In [58]:
# Let's do our usual data science imports
import pandas as pd
import numpy as np
import matplotlib as plot
import sklearn as sk
import importlib
import os
import subprocess


# Define a number of constants
STANDARD_MUTATION = 0.5
STANDARD_MOMENTUM = 0
STANDARD_POPSIZE = 100
STANDARD_SIGMA = 0.004
PATH_TO_LOG: "storage/logs/"
    
EVALUATION_SCHAFFERS = "SchaffersEvaluation"
EVALUATION_KATSUURA = "KatsuuraEvaluation"
EVALUATION_BCF = "BentCigarFunction"
EVALUATION_SPHERE = "SphereEvaluation"

In [2]:
# Define the Setup class
class Setup:
    """A setup for a defined parameter and evaluation, with given parameter values.

    Params:
        - param_name (str): Parameter to define
        - evaluation (str): Name of evaluation function to optimise.
        - parameter_values (list): List of values for the associated parameter
    """
    def __init__(self, param_name = "", evaluation = "", parameter_values = []):
        self.param_name = param_name
        self.evaluation = evaluation
        self.parameter_values = parameter_values
        self.parameter_scores = []

In [59]:
# Define the Simulator class
class Simulator:
    """A simulator for a givenn number of setups, and runs to test on each parameter value of a setup.

    Params:
        - setups(list): A list of Setup to test on
        - runs(int): Number of runs per setup
    """
    
    def __init__(self, setups, runs=10):
        self.setups = setups
        self.runs = runs

    def run(self):
        for setup in self.setups:
            self.runEvalsForSetup(setup)

    def constructParams(self, setup, param):
        """Sets up default and preconfigured parameters."""
        mutation = STANDARD_MUTATION
        momentum = STANDARD_MOMENTUM
        sigma = STANDARD_SIGMA
        popsize = STANDARD_POPSIZE

        if setup.name == "Mutation":
            mutation = param

        if setup.name == "Momentum":
            momentum = param

        if setup.name == "Sigma":
            sigma = param
        
        if setup.name == "Population Size":
            popsize = param

        return {
            'mutation': mutation,
            'momentum': momentum,
            'sigma': sigma,
            'popsize': popsize
        }

    def runEvalsForSetup(self, setup):
        """Runs for the current Setup:setup of parameters to test. """
        for param in setup.parameter_values:
            params = self.constructParams(setup, param)
            self.runEval(setup, params)

    def runEval(self, setup_params ,params):
        """Runs the EA on the igiven eval"""
        scores = 0
        
        for i in range(self.runs):
            cmd = "bash run.sh %s %d %f" % (setup.evaluation, params['popsize'], params['sigma'])
            p = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
            output = p.stdout.readlines()
            scores += output
            retval = p.wait()

In [60]:
cmd = "bash run.sh SchaffersEvaluation 20 0.004"
p = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
output = p.stdout.readlines()

In [61]:
print(output)

[b'[9.270102815118344E-18, 6.147479005770553E-18, 9.376921613434298E-18, 8.46402393066613E-18, 4.60545599183134E-18, 3.1023602364840046E-18, 1.8496490538464617E-17, 3.256494002175242E-18, 9.712430506254235E-18, 4.403209270679601E-18, 5.567197861831922E-18, 1.6451824140677908E-18, 3.4136571275649444E-18, 4.142439954906747E-18, 6.590993314913777E-18, 6.390408995002157E-18, 7.733809723985613E-18, 3.3755124882739404E-18, 4.104446034034295E-18, 1.0741769894622538E-17, 5.904406589790542E-29, 4.2151011951661604E-18, 1.1724018075045083E-17, 1.703179305205848E-17, 1.773920057367517E-17, 8.322691153970681E-17, 1.1019516928003908E-17, 1.0382923740869467E-17, 5.811992283847802E-18, 6.415756450074127E-18, 5.912895198083245E-18, 8.839762403396744E-18, 1.3825989036990626E-17, 5.275305747317451E-17, 2.5901668998871306E-17, 4.858004524112965E-18, 1.0033232264960764E-17, 8.356016014917618E-18, 5.256549134056698E-18, 1.1853738177614125E-17, 8.191331324728901E-65, 2.8898810831097685E-16, 1.569158498024061